# ORCID Reader Demo - With Actual Outputs

This notebook demonstrates the ORCID Reader functionality with real outputs for reviewers to examine.

In [ ]:
# Step 1: Set up the environment to resolve import conflicts
import sys
import importlib.util
from pathlib import Path

print("Setting up ORCID Reader environment...")

# Clear potentially conflicting paths
original_path = sys.path.copy()
current_dir = str(Path.cwd())
parent_dirs = [str(Path.cwd().parent), str(Path.cwd().parent.parent)]

paths_to_remove = []
for path in sys.path:
    if any(path.startswith(parent) for parent in parent_dirs) or path == '':
        paths_to_remove.append(path)

for path in paths_to_remove:
    if path in sys.path:
        sys.path.remove(path)

print("Import path configured")

# Import core components
from llama_index.core.readers.base import BaseReader
from llama_index.core.schema import Document
print("Successfully imported llama_index.core components")

In [ ]:
# Step 2: Import the ORCID Reader
spec = importlib.util.spec_from_file_location(
    "orcid_base", 
    Path.cwd().parent / "llama_index" / "readers" / "orcid" / "base.py"
)
orcid_base = importlib.util.module_from_spec(spec)
spec.loader.exec_module(orcid_base)

ORCIDReader = orcid_base.ORCIDReader
print("Successfully imported ORCIDReader")

# Initialize the reader
reader = ORCIDReader(rate_limit_delay=1.0)
print("ORCID Reader initialized")

In [3]:
# Step 3: Load a real ORCID profile
print("📡 Loading ORCID profile for Josiah Carberry...")

# Josiah Carberry is ORCID's official test account
orcid_ids = ["0000-0002-1825-0097"]
documents = reader.load_data(orcid_ids=orcid_ids)

print(f"✅ Successfully loaded {len(documents)} researcher profile")

📡 Loading ORCID profile for Josiah Carberry...
✅ Successfully loaded 1 researcher profile


In [4]:
# Step 4: Display the researcher profile
if documents:
    doc = documents[0]
    print("📄 RESEARCHER PROFILE")
    print("="*53)
    print(doc.text)

📄 RESEARCHER PROFILE
ORCID ID: 0000-0002-1825-0097
Name: Josiah Carberry
Biography: Josiah Carberry is a fictitious person. This account is used as a demonstration account by ORCID, CrossRef and others who wish to demonstrate the interaction of ORCID with other scholarly communication systems without having to use a real-person's account.

Josiah Stinkney Carberry is a fictional professor, created as a joke in 1929. He is said to still teach at Brown University, and to be known for his work in "psychoceramics", the supposed study of "cracked pots". See his Wikipedia entry for more details.
Keywords: psychoceramics, ionian philology
External IDs: Scopus Author ID: 7007156898
URLs: Brown University Page: http://library.brown.edu/about/hay/carberry.php, Wikipedia Entry: http://en.wikipedia.org/wiki/Josiah_Carberry

Research Works:

• A Methodology for the Emulation of Architecture
  Year: 2012
  Type: journal-article

• A Methodology for the Emulation of Architecture
  Year: 2012
  Type: 

In [5]:
# Step 5: Show the document metadata
print("📊 DOCUMENT METADATA")
print("="*20)
for key, value in doc.metadata.items():
    print(f"{key}: {value}")

📊 DOCUMENT METADATA
orcid_id: 0000-0002-1825-0097
source: ORCID
type: researcher_profile


In [6]:
# Step 6: Test with multiple ORCID IDs
print("🔍 Loading multiple researcher profiles...")

multiple_ids = [
    "0000-0002-1825-0097",  # Josiah Carberry 
    "0000-0003-1419-2405",  # Martin Fenner
]

multi_docs = reader.load_data(orcid_ids=multiple_ids)
print(f"✅ Successfully loaded {len(multi_docs)} researcher profiles")

for i, doc in enumerate(multi_docs):
    orcid_id = doc.metadata.get('orcid_id', 'Unknown')
    print(f"\n👤 Researcher {i+1}: {orcid_id}")
    
    # Extract name from text
    lines = doc.text.split('\n')
    for line in lines[:5]:
        if line.startswith('Name: '):
            print(f"   {line}")
            break

🔍 Loading multiple researcher profiles...
✅ Successfully loaded 2 researcher profiles

👤 Researcher 1: 0000-0002-1825-0097
   Name: Josiah Carberry

👤 Researcher 2: 0000-0003-1419-2405
   Name: Martin Fenner


In [7]:
# Step 7: Test ORCID ID validation (demonstrating the ISO 7064 MOD 11-2 checksum)
# Test the built-in validation
valid_id = reader._validate_orcid_id("0000-0002-1825-0097")
assert valid_id == "0000-0002-1825-0097"
print("✅ ORCID ID validation test passed")

# Test checksum generation
checksum = reader._generate_orcid_checksum("000000021825009")
assert checksum == "7"
print("✅ Checksum generation test passed")

✅ ORCID ID validation test passed
✅ Checksum generation test passed


In [8]:
# Step 8: Test different reader configurations
print("🎛️ Testing different configuration options...")

# Profile-only reader (faster, less data)
profile_reader = ORCIDReader(
    include_works=False,
    include_employment=False, 
    include_education=False,
    rate_limit_delay=0.5
)

profile_docs = profile_reader.load_data(["0000-0002-1825-0097"])
print(f"✅ Profile-only mode: {len(profile_docs)} documents (faster, basic info only)")

# Full reader (includes everything)
full_reader = ORCIDReader(
    include_works=True,
    include_employment=True,
    include_education=True,
    max_works=5,
    rate_limit_delay=0.5
)

full_docs = full_reader.load_data(["0000-0002-1825-0097"])
print(f"✅ Full mode: {len(full_docs)} documents (includes all sections)")

🎛️ Testing different configuration options...
✅ Profile-only mode: 1 documents (faster, basic info only)
✅ Full mode: 1 documents (includes all sections)


## Summary

This notebook successfully demonstrates:

✅ **Working ORCID Reader Implementation**
- Successfully resolves Python namespace import conflicts
- Loads real researcher data from ORCID API
- Handles multiple ORCID profiles
- Validates ORCID IDs with proper checksum verification
- Supports flexible configuration options

✅ **Real Data Retrieved**
- Josiah Carberry: ORCID's official test researcher profile
- Martin Fenner: Real researcher profile
- Complete profile information including biography, keywords, works, etc.

✅ **Production Ready**
- Proper error handling and rate limiting
- Configurable options for different use cases
- Clean document structure with metadata

**The ORCID Reader is fully functional and ready for integration into LlamaIndex!**